### Deliverable 1: Preprocessing the Data for a Neural Network

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
file_path = "charity_data.csv"
application_df = pd.read_csv(file_path)
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
#  YOUR CODE GOES HERE

In [ ]:
application_df.head()

In [ ]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values
#  YOUR CODE GOES HERE

In [ ]:
unique_values = unique_values.loc[unique_values > 10]
unique_values

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
Application_type_df = application_df.APPLICATION_TYPE.value_counts()
Application_type_df
#  YOUR CODE GOES HERE

In [ ]:
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
Application_type_df.plot.density()
#  YOUR CODE GOES HERE

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_values = list(Application_type_df[Application_type_df < 150].index)
# REplace the DataFrame
for values in replace_values:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(values, "Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
CLASSIFICATION_type_df = application_df.CLASSIFICATION.value_counts()
CLASSIFICATION_type_df
#  YOUR CODE GOES HERE

In [ ]:
# Visualize the value counts of CLASSIFICATION
CLASSIFICATION_type_df.plot.density()
#  YOUR CODE GOES HERE

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_values2 = list(CLASSIFICATION_type_df[CLASSIFICATION_type_df < 1880].index)
# REplace the DataFrame
for values in replace_values2:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(values, "Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application
#  YOUR CODE GOES HERE

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application]))
#  YOUR CODE GOES HERE

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application)
encode_df.head(10)

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application, 1)
application_df.head()
#  YOUR CODE GOES HERE

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis=1).values

#  YOUR CODE GOES HERE

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

#  YOUR CODE GOES HERE

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[1])
hidden_nodes_layer1 = 7

#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation= "linear")
)

#  YOUR CODE GOES HERE

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="linear"))
#  YOUR CODE GOES HERE

# Check the structure of the model
nn.summary()

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 5)
#  YOUR CODE GOES HERE

In [ ]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=10, callbacks=[cp_callback])
#  YOUR CODE GOES HERE

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
nn.save("trained_application.h5")